## 파일로 로딩

In [1]:
from transformers import AutoTokenizer, BartForConditionalGeneration

task_name = "rationale"
model_path = f"/home/jovyan/hdfs-jmt-rungjoo-private/save_models/facet/multi_task/{task_name}"

model = BartForConditionalGeneration.from_pretrained(model_path)        
model = model.cuda()
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/jovyan/workspace-main/venv_main/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
with open("../../data/test.json", 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [13]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
test_result = {}
for k, data in test_data.items():
    query = data['query']
    options_overall_label = data['options_overall_label']
    gt_facet_list = data['facet']

    f_string = f"[facet] {query}"

    inputs = tokenizer(f_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt', add_special_tokens=False)
    inputs.to(device)    

    token_ids = model.generate(inputs["input_ids"], max_length=tokenizer.model_max_length)
    pred_facet_string = tokenizer.decode(token_ids[0], skip_special_tokens=True)
    pred_facet_list = [x.strip() for x in pred_facet_string.split("|")]

    test_result[k] = {}
    test_result[k]['query'] = query
    test_result[k]['pred'] = [x for x in pred_facet_list if x.strip() != ""]
    test_result[k]['label'] = gt_facet_list
    test_result[k]['options_overall_label'] = options_overall_label        
    
    break

In [15]:
inputs

{'input_ids': tensor([[50265,  6056,   293,  2726,    23,   462, 26970,   343]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [16]:
f_string = f"[facet] {query}"
tokenizer(f_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt', add_special_tokens=False)

{'input_ids': tensor([[50265,  6056,   293,  2726,    23,   462, 26970,   343]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
f_string = f"[rationale] {query}"
tokenizer(f_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt', add_special_tokens=False)

{'input_ids': tensor([[50268,  6056,   293,  2726,    23,   462, 26970,   343]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}